In [1]:
#Import dependencies
import requests
import json
import pandas as pd
from pprint import pprint
API_KEY = "****************************"



# Searching without using Fred-full Library

In [3]:
url = "https://api.stlouisfed.org/fred/series"
params = {
    "api_key": API_KEY,
    "file_type": "json",
    "series_id": "MSPUS"
}
response = requests.get(url=url, params=params)
response.status_code
data = response.json()
data

{'realtime_start': '2023-11-09',
 'realtime_end': '2023-11-09',
 'seriess': [{'id': 'MSPUS',
   'realtime_start': '2023-11-09',
   'realtime_end': '2023-11-09',
   'title': 'Median Sales Price of Houses Sold for the United States',
   'observation_start': '1963-01-01',
   'observation_end': '2023-07-01',
   'frequency': 'Quarterly',
   'frequency_short': 'Q',
   'units': 'Dollars',
   'units_short': '$',
   'seasonal_adjustment': 'Not Seasonally Adjusted',
   'seasonal_adjustment_short': 'NSA',
   'last_updated': '2023-10-25 09:01:02-05',
   'popularity': 90}]}

In [2]:
!pip install full-fred

# Searching for Series using Full Fred library


In [4]:
from full_fred.fred import Fred
fred = Fred('api_key.txt')

fred.get_api_key_file()

'api_key.txt'

In [31]:
# Search through Fred database for series 
# Open in text editor to go through data 
# Command + f to search through text editor

# #Storing in variable to avoid running output
home_sales_search = fred.search_for_series(search_words=["home sales"])

# Grab the 'id' value for the series you want
# For example - if we want the id from the first dataset
#id = 'MSPUS' 

In [6]:
# Use the .get_series_df method and input the 'id' from the desired dataset
median_home_price = fred.get_series_df('MSPUS')
median_home_price

,realtime_start,realtime_end,date,value
0,2023-11-09,2023-11-09,1963-01-01,17800
1,2023-11-09,2023-11-09,1963-04-01,18000
2,2023-11-09,2023-11-09,1963-07-01,17900
3,2023-11-09,2023-11-09,1963-10-01,18500
4,2023-11-09,2023-11-09,1964-01-01,18500
...,...,...,...,...
238,2023-11-09,2023-11-09,2022-07-01,468000.0
239,2023-11-09,2023-11-09,2022-10-01,479500.0
240,2023-11-09,2023-11-09,2023-01-01,429000.0
241,2023-11-09,2023-11-09,2023-04-01,418500.0


In [29]:
#Create pandas Dataframe
median_home_price_df = pd.DataFrame(median_home_price)

#Rename Columns
median_home_price_df = median_home_price_df.rename(columns={
    "date": "Date",
    "value": "Median Home Sale Price"
})

#Grab the columns needed for dataframe
median_home_price_df = median_home_price_df[["Date", "Median Home Sale Price"]]

#Change the 'Median Home Sale Price' column to float
median_home_price_df["Median Home Sale Price"] = median_home_price_df["Median Home Sale Price"].astype(float)

#Display Dataframe
median_home_price_df.head()

,Date,Median Home Sale Price
0,1963-01-01,17800.0
1,1963-04-01,18000.0
2,1963-07-01,17900.0
3,1963-10-01,18500.0
4,1964-01-01,18500.0


In [30]:
interest_rate_search = fred.search_for_series(search_words=["interest rate"])


In [7]:
mortgage_30 = fred.get_series_df("MORTGAGE30US")
mortgage_30

#Data sourced from '30-Year Fixed Rate Mortgage Average in the United States'

,realtime_start,realtime_end,date,value
0,2023-11-09,2023-11-09,1971-04-02,7.33
1,2023-11-09,2023-11-09,1971-04-09,7.31
2,2023-11-09,2023-11-09,1971-04-16,7.31
3,2023-11-09,2023-11-09,1971-04-23,7.31
4,2023-11-09,2023-11-09,1971-04-30,7.29
...,...,...,...,...
2741,2023-11-09,2023-11-09,2023-10-12,7.57
2742,2023-11-09,2023-11-09,2023-10-19,7.63
2743,2023-11-09,2023-11-09,2023-10-26,7.79
2744,2023-11-09,2023-11-09,2023-11-02,7.76


In [8]:
mortgage_15 = fred.get_series_df("MORTGAGE15US")
mortgage_15
#Data sourced from '15-Year Fixed Rate Mortgage Average in the United States'

,realtime_start,realtime_end,date,value
0,2023-11-09,2023-11-09,1991-08-30,8.77
1,2023-11-09,2023-11-09,1991-09-06,8.79
2,2023-11-09,2023-11-09,1991-09-13,8.68
3,2023-11-09,2023-11-09,1991-09-20,8.62
4,2023-11-09,2023-11-09,1991-09-27,8.59
...,...,...,...,...
1676,2023-11-09,2023-11-09,2023-10-12,6.89
1677,2023-11-09,2023-11-09,2023-10-19,6.92
1678,2023-11-09,2023-11-09,2023-10-26,7.03
1679,2023-11-09,2023-11-09,2023-11-02,7.03


In [11]:
home_price_search = fred.search_for_series(search_words=["home price"])

In [9]:
home_price_index = fred.get_series_df("CSUSHPINSA")
home_price_index

#Dataframe sourced from S&P/Case-Shiller U.S. National Home Price Index

,realtime_start,realtime_end,date,value
0,2023-11-09,2023-11-09,1975-01-01,.
1,2023-11-09,2023-11-09,1975-02-01,.
2,2023-11-09,2023-11-09,1975-03-01,.
3,2023-11-09,2023-11-09,1975-04-01,.
4,2023-11-09,2023-11-09,1975-05-01,.
...,...,...,...,...
579,2023-11-09,2023-11-09,2023-04-01,301.469
580,2023-11-09,2023-11-09,2023-05-01,305.427
581,2023-11-09,2023-11-09,2023-06-01,308.307
582,2023-11-09,2023-11-09,2023-07-01,310.16200000000003


In [12]:
gdp_search = fred.search_for_series(search_words=["GDP"])

In [10]:
household_debt_to_gdp = fred.get_series_df("HDTGPDUSQ163N")
household_debt_to_gdp

,realtime_start,realtime_end,date,value
0,2023-11-09,2023-11-09,2005-01-01,86.7953478741027
1,2023-11-09,2023-11-09,2005-04-01,89.3665332228971
2,2023-11-09,2023-11-09,2005-07-01,92.1382753543162
3,2023-11-09,2023-11-09,2005-10-01,94.4367215780109
4,2023-11-09,2023-11-09,2006-01-01,91.7366889603057
...,...,...,...,...
69,2023-11-09,2023-11-09,2022-04-01,76.1400500108485
70,2023-11-09,2023-11-09,2022-07-01,77.4079771917544
71,2023-11-09,2023-11-09,2022-10-01,78.2792851609939
72,2023-11-09,2023-11-09,2023-01-01,74.35503339374
